In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [4]:
!ls

sample_data  spark-3.4.0-bin-hadoop3  spark-3.4.0-bin-hadoop3.tgz


In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.appName("Olympic Data Processing").config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.18.0").getOrCreate()
spark

In [6]:
%pip install -U pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [11]:

import pyspark.pandas as ps

# Lecture des données en utilisant Spark
df_athletes = ps.read_json('/content/content/data/input/olympic_athletes.json')
xml_file_path = '/content/content/data/input/olympic_hosts.xml'
df_hosts = spark.read.format("xml").option("rowTag", "row").load(xml_file_path).toPandas().set_index('index')
df_results = ps.read_html('/content/content/data/input/olympic_results.html',index_col=0)[0]

/content/spark-3.4.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [12]:
df_results.head()

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,None,None,None,None
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,None,None,None,None
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,None,None,None,None
3,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,None,None,None,None
4,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,None,None,None,None


In [13]:
df_hosts

,game_end_date,game_location,game_name,game_season,game_slug,game_start_date,game_year
index,,,,,,,
0,2022-02-20 12:00:00,China,Beijing 2022,Winter,beijing-2022,2022-02-04 15:00:00,2022
1,2021-08-08 14:00:00,Japan,Tokyo 2020,Summer,tokyo-2020,2021-07-23 11:00:00,2020
2,2018-02-25 08:00:00,Republic of Korea,PyeongChang 2018,Winter,pyeongchang-2018,2018-02-08 23:00:00,2018
3,2016-08-21 21:00:00,Brazil,Rio 2016,Summer,rio-2016,2016-08-05 12:00:00,2016
4,2014-02-23 16:00:00,Russian Federation,Sochi 2014,Winter,sochi-2014,2014-02-07 04:00:00,2014
5,2012-08-12 19:00:00,Great Britain,London 2012,Summer,london-2012,2012-07-27 07:00:00,2012
6,2010-02-28 04:00:00,Canada,Vancouver 2010,Winter,vancouver-2010,2010-02-12 16:00:00,2010
7,2008-08-24 12:00:00,China,Beijing 2008,Summer,beijing-2008,2008-08-08 00:00:00,2008
8,2006-02-26 19:00:00,Italy,Turin 2006,Winter,turin-2006,2006-02-10 07:00:00,2006


In [14]:
def format_athletes_unique_format(row):
  if ps.notna(row['athlete_full_name']):
    row['athletes'] = f"({row['athlete_full_name']}, {row['athlete_url']})"
  return row
df_results = df_results.apply(format_athletes_unique_format, axis=1)
df_results = df_results.drop(['athlete_full_name', 'athlete_url'], axis=1)
df_results.head(10)

/content/spark-3.4.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,None,None
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,None,None
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,None,None
3,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,None,None
4,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,None,None
5,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Zuzana HAJKOVA', 'https://olympics.com/en/a...",False,6,Czech Republic,CZ,CZE,None,None
6,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jenny PERRET', 'https://olympics.com/en/ath...",False,7,Switzerland,CH,SUI,None,None
7,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Vicky PERSINGER', 'https://olympics.com/en/...",False,8,United States of America,US,USA,None,None
8,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Suyuan FAN', 'https://olympics.com/en/athle...",False,9,People's Republic of China,CN,CHN,None,None
9,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Tahli GILL', 'https://olympics.com/en/athle...",False,10,Australia,AU,AUS,None,None


In [15]:
def get_type_by_event_gender(row):
    row['event_gender'] = 'Men'
    if "women" in row['event_title'].lower() or "ladi" in row['event_title'].lower():
        row['event_gender'] = 'Women'
    elif "mixed" in row['event_title'].lower():
        row['event_gender'] = 'Mixed'

    return row

def format_unit(row):
    if (row['value_type'] == 'DISTANCE' and isinstance(row['value_unit'], str) and 'w' in row['value_unit']):
        row['value_unit'] = row['value_unit'].replace('w', '')
    elif row['value_type'] == 'TIME' and isinstance(row['value_unit'], str):
      try:
        total_milliseconds = int(row['value_unit'])
        milliseconds = total_milliseconds % 1000
        total_seconds = total_milliseconds // 1000
        seconds = total_seconds % 60
        total_minutes = total_seconds // 60
        minutes = total_minutes % 60
        hours = total_minutes // 60
        row['value_unit'] = f"{hours:02}:{minutes:02}:{seconds:02}:{milliseconds:03}"
      except:
        row['value_unit'] = row['value_unit']

    return row

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

df_results = df_results.apply(get_type_by_event_gender, axis=1)
df_results = df_results.apply(format_unit, axis=1)
df_results.head()

/content/spark-3.4.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/content/spark-3.4.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,value_unit,value_type,event_gender
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,None,None,Mixed
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,None,None,Mixed
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,None,None,Mixed
3,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,None,None,Mixed
4,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,None,None,Mixed


In [18]:
df_results

,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,value_unit,value_type,event_gender
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,None,None,Mixed
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,None,None,Mixed
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,None,None,Mixed
3,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,None,None,Mixed
4,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,None,None,Mixed
5,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Zuzana HAJKOVA', 'https://olympics.com/en/a...",False,6,Czech Republic,CZ,CZE,None,None,Mixed
6,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jenny PERRET', 'https://olympics.com/en/ath...",False,7,Switzerland,CH,SUI,None,None,Mixed
7,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Vicky PERSINGER', 'https://olympics.com/en/...",False,8,United States of America,US,USA,None,None,Mixed
8,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Suyuan FAN', 'https://olympics.com/en/athle...",False,9,People's Republic of China,CN,CHN,None,None,Mixed
9,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Tahli GILL', 'https://olympics.com/en/athle...",False,10,Australia,AU,AUS,None,None,Mixed


In [21]:
# Écriture des données
df_hosts.to_csv('/content/content/data/output/hosts.csv', header=True)
df_results.to_csv('/content/content/data/output/results_jo.csv', header=True)
df_athletes.to_csv('/content/content/data/output/athletes.csv', header=True)

# Arrêt de la session Spark
spark.stop()